<p align="center">
  <img src="./UFC.jpg" alt="Texto alternativo" />
</p>


<h1 align="center">Análisis exploratorio: Métricas de la UFC</h1>
&nbsp; 


Este **EDA** se centra principalmente en abordar las hipótesis más comúnmente planteadas por los *"opinólogos de bar"* 🍻 en el mundo de los deportes de contacto 🥋.

Ha sido realizada a través de la exploración de 2 **DATASETS** 📊 encontrados en [Kaggle](https://www.kaggle.com/)
repletos de estadísticas específicas 📈 y haciendo **WebsCrapping** 🔍 para sacar alguna variable concreta más.

Como conclusión **ofrecerá una perspectiva** más clara de **pasos a seguir** para llegar a ser campeón de la UFC🏆🏆.

&nbsp; 

#### A lo largo del análisis, podrás ir visualizando los resultados de las **hipótesis** que han planteado esta investigación y el proceso de resolución de la misma:

---

##### [1. ¿Qué relación hay entre la **guardia** y el Éxito en la carrera? 🤼](#Guardia)

##### [2. ¿Qué probabilidad hay de **ganar** respecto a una buena **defensa de derribo**? 🥋](#Defensa)

##### [3. ¿Cómo afecta la **edad** en un enfrentamiento? 👵👦](#Edad)

##### [4. ¿Es más probable ganar por **KO** en divisiones de **peso pesado** respecto a las demás? 💪⚖️](#KO's)

##### [5. ¿Cómo influye la **altura** y el alcance? 📏](#Altura)

---


&nbsp;







Ántes que nada, aquí una lista de las librerías que se han ido necesitando

In [135]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

Los **datasets recopilados** inicialmente no vienen con los datos estructurados y limpios como se necesitan para abordar las hipotésis.  
Se procede a explicar los pasos seguidos:


### **Data set:** Estadísticas de los peleadores

In [136]:
#Leyendo DataSet de Peleadores
estadisticas_peleador = pd.read_csv("../data/Estadisticas_peleador/estadisticas_peleador.csv")
fuente_1 = "https://www.kaggle.com/datasets/maksbasher/ufc-complete-dataset-all-events-1996-2024/data"
estadisticas_peleador.head(5)

,name,wins,losses,height,weight,reach,stance,age,SLpM,sig_str_acc,SApM,str_def,td_avg,td_acc,td_def,sub_avg
0,Amanda Ribas,12.0,5.0,160.02,56.70,167.64,Orthodox,30.0,4.63,0.40,3.40,0.61,2.07,0.51,0.85,0.7
1,Rose Namajunas,13.0,6.0,165.10,56.70,165.10,Orthodox,31.0,3.69,0.41,3.51,0.63,1.38,0.47,0.59,0.5
2,Karl Williams,10.0,1.0,190.50,106.59,200.66,Orthodox,34.0,2.87,0.52,1.70,0.60,4.75,0.50,1.00,0.2
3,Justin Tafa,7.0,4.0,182.88,119.75,187.96,Southpaw,30.0,4.09,0.54,5.02,0.47,0.00,0.00,0.50,0.0
4,Edmen Shahbazyan,13.0,4.0,187.96,83.91,190.50,Orthodox,26.0,3.60,0.52,4.09,0.45,2.24,0.38,0.63,0.6


In [137]:
#Comprobando los Missing-Values
estadisticas_peleador.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         2478 non-null   object 
 1   wins         2478 non-null   float64
 2   losses       2478 non-null   float64
 3   height       2478 non-null   float64
 4   weight       2478 non-null   float64
 5   reach        1823 non-null   float64
 6   stance       2401 non-null   object 
 7   age          2318 non-null   float64
 8   SLpM         2478 non-null   float64
 9   sig_str_acc  2478 non-null   float64
 10  SApM         2478 non-null   float64
 11  str_def      2478 non-null   float64
 12  td_avg       2478 non-null   float64
 13  td_acc       2478 non-null   float64
 14  td_def       2478 non-null   float64
 15  sub_avg      2478 non-null   float64
dtypes: float64(14), object(2)
memory usage: 310.0+ KB


In [138]:
#Creando la copia:
df_1 = estadisticas_peleador.copy()

#### Para mayor compresión del análisis se iran **traduciendo** los datasets.

In [139]:
#Diccionarios de traducción:

ingles_columnas1 = ['name', 'wins', 'losses', 'height', 'weight', 'reach', 'stance', 'age',
                    'SLpM', 'sig_str_acc', 'SApM', 'str_def', 'td_avg', 'td_acc', 'td_def',
                    'sub_avg']

español_columnas1= ["Nombre","Victorias","Derrotas","Altura","Peso","Alcance",
                    "Guardia","Edad","Golpes_min","Golpes_acer","Golpes_abs",
                    "Golpes_def","Derribos_med","Derribos_acer", "Derribos_def",
                    "Sumision_int"]

guardia_dict = {'Orthodox':'Ortodoxo', 'Southpaw':'Zurdo', 'Switch':'Ambidiestro', 'Open Stance':'Guardia Baja'}

In [140]:
#Traduciendo:
df_1.rename(columns=dict(zip(ingles_columnas1, español_columnas1)), inplace=True)
df_1['Guardia'] = df_1['Guardia'].replace({**guardia_dict})

In [141]:
#Eliminando las filas que contengan algún missing-values:
df_1.dropna(subset=['Alcance', 'Guardia', 'Edad'], inplace=True)


In [142]:
#Comprobando si hay peleadores duplicados:
peleadores_duplicados = df_1['Nombre'].duplicated().sum()
df_1[df_1['Nombre'].duplicated(keep= False)]

,Nombre,Victorias,Derrotas,Altura,Peso,Alcance,Guardia,Edad,Golpes_min,Golpes_acer,Golpes_abs,Golpes_def,Derribos_med,Derribos_acer,Derribos_def,Sumision_int
382,Bruno Silva,23.0,10.0,182.88,83.91,187.96,Ortodoxo,34.0,4.43,0.51,5.33,0.43,1.09,0.27,0.73,0.0
638,Bruno Silva,13.0,5.0,162.56,56.70,165.10,Ortodoxo,34.0,3.05,0.46,3.09,0.59,2.57,0.30,0.64,0.3
1879,Edward Faaloloto,2.0,5.0,185.42,70.31,177.80,Ortodoxo,39.0,2.28,0.32,6.25,0.44,1.01,0.25,0.33,0.0
1897,Edward Faaloloto,2.0,5.0,177.80,70.31,177.80,Ortodoxo,39.0,2.28,0.32,6.25,0.44,1.01,0.25,0.33,0.0


El peleador **Bruno silva** se actualizó con mayor número de victorias asi que debemos borrar la fila antigua.  

Edward **Faaloloto** simplemente hubo una confusión con su altura, se queda el valor mas alto.

In [143]:
#Eliminando peleadores duplicados:
df_1.drop([638, 1897], inplace=True)

La **nacionalidad** de los peleadores no estaba definida en el dataset, así que se procederá a hacer **WebsCrapping** de la *"wikipedia"*.  

La **función principal** quedará encapsulada en una clase en [MainWebscrapping.py](../utils/MainWebscrapping.py) y función de *"obtener_nacionalidad"* en [Webscrapping.py](../utils/Webscrapping.py).

Se generara un ***".csv"*** con los luchadores y su nacionalidad.

In [144]:
#Leyendo csv:
nacionalidad_peleador = pd.read_csv("../data/Nacionalidad_peleador/nacionalidades_luchadores.csv")
nacionalidad_peleador.head()

,Luchador,Nacionalidad
0,Amanda Ribas,Brasileña
1,Rose Namajunas,EstadounidenseLituana
2,Karl Williams,missingvalue
3,Justin Tafa,Neozelandés
4,Edmen Shahbazyan,Estadounidense


In [145]:
#Insertando columna "Nacionalidad" 
nacionalidad = nacionalidad_peleador.iloc[:, 1]  
estadisticas_peleador = df_1.insert(1, 'Pais', nacionalidad)

In [146]:
# Eliminar filas con NaN solo en la columna 'Nacionalidad'
df_1_clean = df_1.dropna(subset=['Pais'])
df_1_clean= df_1_clean[~df_1_clean['Pais'].str.contains('missingvalue', na=False)] 
   

Los países no estan normalizados, se utilizará un diccionario para corregir ese error.

In [147]:
# Diccionario paises normalizados
pais_dict = {'estado': 'Estados Unidos', 'brasi': 'Brasil', 'mexic': 'México', 'finlan': 'Finlandia','pana': 'Panamá',
            'cuba': 'Cuba', 'esp': 'España', 'arme': 'Armenia', 'nige': 'Nigeria','rus': 'Rusia','uzb': 'Uzbekistán',
            'ingl': 'Inglaterra','chi': 'China', 'neoze': 'Nueva Zelanda', 'suda': 'Sudáfrica', 'marr': 'Marruecos',
            'austr': 'Australia','pola': 'Polonia','sue': 'Suecia','isra': 'Israel','gale': 'Gales','ecua': 'Ecuador',
            'brit': 'Inglaterra','esco': 'Escocia','ucra': 'Ucrania','suri': 'Surinam','fili': 'Filipinas','canad': 'Canadá',
            'afg': 'Afganistán','argen': 'Argentina','peru': 'Perú','tay': 'Tayikistán','georg': 'Georgia','jap': 'Japón',
            'fran': 'Francia','ita': 'Italia','ira': 'Irak','puerto': 'Puerto Rico','urug': 'Uruguay','tún': 'Túnez',
            'rum': 'Rumanía','let': 'Letonia','neerl': 'Holanda','irla': 'Irlanda','belg': 'Bélgica','costar': 'Costa Rica',
            'portu': 'Portugal','cro': 'Croacia','grie': 'Grecia','dane': 'Dinamarca','boliv': 'Bolivia','liban': 'Líbano',
            'kaz': 'Kazajistán','surcor': 'Corea del Sur','cameru': 'Camerún','bosn': 'Bosnia','guya': 'Guyana',
            'búlg': 'Bulgaria','colom': 'Colombia','isla': 'Islandia','tail': 'Tailandia', 'litu': 'Lituania', 
            'ale': 'Alemania','suiz': 'Suiza','serb': 'Serbia','congo': 'Congo','ghan': 'Ghana','molda': 'Moldavia',
            'kirg': 'Kirguistán','jama': 'Jamaica','holand': 'Holanda'}

In [148]:
#Remplazando Países normalizados:

def reemplazar_pais(pais):
    pais_lower = pais.lower() 
    for clave, nombre in pais_dict.items():
        if clave in pais_lower:
            return nombre
    return pais  

df_1_clean['Pais'] = df_1_clean['Pais'].apply(reemplazar_pais)

#print(f"Paises únicos: {df_1_clean['Pais'].unique()}")


In [149]:
#Añadiento Países normalizados al DataFrame
df_1_clean['Pais'] = df_1_clean['Pais'].apply(reemplazar_pais)

In [150]:
#Reseteamos el indice:
df_1_clean.reset_index(drop=True, inplace=True)


In [151]:
#Guardando DataFrame en csv:
#df_1_clean.to_csv("../data/Estadisticas_peleador/Estadisticas_peleador_limpio.csv")


In [152]:
#Estadisticas_peleador_limpio:
df_1_clean.head()

,Nombre,Pais,Victorias,Derrotas,Altura,Peso,Alcance,Guardia,Edad,Golpes_min,Golpes_acer,Golpes_abs,Golpes_def,Derribos_med,Derribos_acer,Derribos_def,Sumision_int
0,Amanda Ribas,Brasil,12.0,5.0,160.02,56.70,167.64,Ortodoxo,30.0,4.63,0.40,3.40,0.61,2.07,0.51,0.85,0.7
1,Rose Namajunas,Estados Unidos,13.0,6.0,165.10,56.70,165.10,Ortodoxo,31.0,3.69,0.41,3.51,0.63,1.38,0.47,0.59,0.5
2,Justin Tafa,Nueva Zelanda,7.0,4.0,182.88,119.75,187.96,Zurdo,30.0,4.09,0.54,5.02,0.47,0.00,0.00,0.50,0.0
3,Edmen Shahbazyan,Estados Unidos,13.0,4.0,187.96,83.91,190.50,Ortodoxo,26.0,3.60,0.52,4.09,0.45,2.24,0.38,0.63,0.6
4,Payton Talbott,Estados Unidos,8.0,0.0,177.80,61.23,177.80,Ambidiestro,25.0,8.05,0.54,3.58,0.51,0.00,0.00,0.90,0.5


### Variables totales:

- **Peleadores**: 694 🥊

- **Métricas**: 17 📒

&nbsp; 

### Continuamos con la **limpieza** del siguiente set de datos ***"Enfrentamientos"***

In [153]:
#Leyendo DataSet de Enfrentamientos
enfrentamientos = pd.read_csv("../data/Enfrentamientos/Enfrentamientos.csv")
fuente_2 = "https://www.kaggle.com/datasets/maksbasher/ufc-complete-dataset-all-events-1996-2024/data"
enfrentamientos.head(5)

,event,date,location,r_fighter,b_fighter,status,r_kd,b_kd,r_str,b_str,r_td,b_td,r_sub,b_sub,weight_class,method,method_detailed,round,time
0,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Rose Namajunas,Amanda Ribas,win,0.0,0.0,93.0,83.0,1.0,4.0,0.0,0.0,Women's Flyweight,U-DEC,NaN,5.0,5:00
1,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Karl Williams,Justin Tafa,win,0.0,0.0,40.0,21.0,7.0,0.0,1.0,0.0,Heavyweight,U-DEC,NaN,3.0,5:00
2,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Edmen Shahbazyan,AJ Dobson,win,1.0,0.0,27.0,15.0,1.0,0.0,0.0,0.0,Middleweight,KO/TKO,Punches,1.0,4:33
3,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Payton Talbott,Cameron Saaiman,win,1.0,0.0,79.0,31.0,0.0,0.0,0.0,0.0,Bantamweight,KO/TKO,Punches,2.0,0:21
4,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Youssef Zalal,Billy Quarantillo,win,0.0,0.0,33.0,10.0,2.0,0.0,2.0,0.0,Featherweight,SUB,Rear Naked Choke,2.0,1:50


In [154]:
#Verificando los missing_values:
enfrentamientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7582 entries, 0 to 7581
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   event            7497 non-null   object 
 1   date             7497 non-null   object 
 2   location         7497 non-null   object 
 3   r_fighter        7497 non-null   object 
 4   b_fighter        7497 non-null   object 
 5   status           7582 non-null   object 
 6   r_kd             7476 non-null   float64
 7   b_kd             7476 non-null   float64
 8   r_str            7476 non-null   float64
 9   b_str            7476 non-null   float64
 10  r_td             7476 non-null   float64
 11  b_td             7476 non-null   float64
 12  r_sub            7476 non-null   float64
 13  b_sub            7476 non-null   float64
 14  weight_class     7497 non-null   object 
 15  method           7497 non-null   object 
 16  method_detailed  3836 non-null   object 
 17  round         

In [155]:
#Creandp copia:
df_2 = enfrentamientos.copy()

In [156]:
#Eliminando columnas que no serán objeto de estudio:
df_2.drop(['status','r_kd','b_kd', 'r_str', 'b_str', 'r_td', 'b_td', 'r_sub', 'b_sub'], axis=1, inplace=True)

In [157]:
#Eliminando cualquier fila que tenga un solo missingvalue:
df_2.dropna(inplace=True)


Se cambiarán todas las filas y columnas que estén en ingles para mayor comprensión a la hora de trabajarlas

In [158]:
#Importando diccionario de traducciones:

#Diccionario de columnas
ingles_columnas2 = ["event","date","location","r_fighter",
                    "b_fighter","weight_class","method",
                    "method_detailed","round","time"]

español_columnas2= ["Evento","Día","Donde",
                    "Ganador","Perdedor","Division",
                    "Metodo","Detalles","Round","Tiempo"]

#Todas las divisiones
division_dict = {'Middleweight': 'Peso Medio',
                'Bantamweight': 'Peso Gallo',
                'Featherweight': 'Peso Pluma',
                'Heavyweight': 'Peso Pesado',
                "Women's Bantamweight": 'Peso Gallo',
                'Lightweight': 'Peso Ligero',
                'Flyweight': 'Peso Mosca',
                "Women's Strawweight": 'Peso Paja',
                'Welterweight': 'Peso Welter',
                'Light Heavyweight': 'Peso Semipesado',
                "Women's Flyweight": 'Peso Mosca',
                'Catch Weight': 'Peso Pactado',
                "Women's Featherweight": 'Peso Pluma',
                'Super Heavyweight': 'Peso Super Pesado',
                'Open Weight': 'Peso Abierto'}

#Técnicas de Golpeo:
dict_golpes = {"Punches":"Golpes",
                "Kick":"Patada",
                "Punch":"Puño",
                "Elbows":"Codos",
                "Elbow":"Codo",
                "Flying Knee":"Rodilla Voladora",
                "Knees":"Rodillas",
                "Knee":"Rodilla",
                "Kicks":"Patadas",
                "Spinning Back Kick":"Patada Circular",
                "Spinning Back Fist":"Puñetazo Circular",
                "Spinning Back Elbow":"Codazo Circular",
                "Headbutts":"Cabezazos",
                "Slam":"Golpe Bajo"}

#Técnicas de Grappling y Estrangulaciones
dict_suelo = {"Rear Naked Choke":"Estrangulación Trasera",
                "D'Arce Choke":"Estrangulación D'Arce",
                "Guillotine Choke":"Estrangulación Guillotina",
                "Arm Triangle":"Triángulo de Brazo",
                "Armbar":"Llave de Brazo",
                "Triangle Choke":"Estrangulación Triangular",
                "Anaconda Choke":"Estrangulación Anaconda",
                "Neck Crank":"Torcedura de Cuello",
                "Kimura":"Llave Kimura",
                "Kneebar":"Llave de Rodilla",
                "Inverted Triangle" : "Triángulo Invertido",
                "Von Flue Choke" : "Estrangulación Von Flue",
                "Scarf Hold" : "Sujeción de Bufanda",
                "Straight Armbar" : "Llave de Brazo Recto",
                "Heel Hook" : "Llave de Talón",
                "Ankle Lock" : "Llave de Tobillo",
                "Forearm Choke" : "Estrangulación con Antebrazo",
                "Ezekiel Choke" : "Estrangulación Ezekiel",
                "Peruvian Necktie" : "Corbata Peruana",
                "Schultz Front Headlock" : "Llave de Cabeza Schultz",
                "Bulldog Choke" : "Estrangulación Bulldog",
                "Suloev Stretch" : "Estiramiento Suloev",
                "Omoplata" : "Omoplato",
                "Calf Slicer" : "Cortador de Pantorrilla",
                "North-South Choke" : "Estrangulación Norte-Sur",
                "Pace/Pillory Choke" : "Estrangulación Pace/Pillory",
                "Toe Hold" : "Llave Dedo del Pie",
                "Shoulder Choke" : "Estrangulación de Hombro",
                "Twister":"Torcedura"}

traduccion_dict = {**dict_golpes,**dict_suelo}

In [159]:
#Implementando los diccionarios:
df_2.rename(columns=dict(zip(ingles_columnas2, español_columnas2)), inplace=True)
df_2['Detalles'] = df_2['Detalles'].replace(traduccion_dict)
df_2['Division'] = df_2['Division'].replace({**division_dict})

In [160]:
#Voy a elimnar categorías no influyentes en el estudio
eliminar = ["Other","Other - Lock", "Other - Choke" ,"injury", "Gi Choke", "Keylock"]
eliminados = '|'.join(eliminar)
df_2 = df_2[~df_2['Detalles'].str.contains(eliminados, regex=True)]


In [161]:
#Reseteando índice:
df_2.reset_index(drop=True, inplace=True)



In [162]:
#Guardando DataFrame en csv.
#df_2.to_csv("../data/Enfrentamientos/Enfrentamientos_limpio.csv")



In [163]:
#Enfrentamientos_limpios:
df_2.head()

,Evento,Día,Donde,Ganador,Perdedor,Division,Metodo,Detalles,Round,Tiempo
0,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Edmen Shahbazyan,AJ Dobson,Peso Medio,KO/TKO,Golpes,1.0,4:33
1,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Payton Talbott,Cameron Saaiman,Peso Gallo,KO/TKO,Golpes,2.0,0:21
2,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Youssef Zalal,Billy Quarantillo,Peso Pluma,SUB,Estrangulación Trasera,2.0,1:50
3,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Fernando Padilla,Luis Pajuelo,Peso Pluma,SUB,Estrangulación D'Arce,1.0,2:45
4,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Julian Erosa,Ricardo Ramos,Peso Pluma,SUB,Estrangulación Guillotina,1.0,2:15


### Variables totales:

- **Enfrentamientos**: 3815 🥊

- **Métricas**: 10 📒

&nbsp; 

Se creará un DataFrame de Divisiones y sus respectivos rangos de peso

In [164]:
#Introduciendo diccionario de divisiones:

informacion = {'Division':['Peso Medio','Peso Gallo','Peso Pluma','Peso Pesado','Peso Gallo','Peso Ligero',
                           'Peso Mosca','Peso Paja','Peso Welter','Peso Semipesado',
                           'Peso Pactado','Peso Super Pesado','Peso Abierto'],
               'Peso (kg)':['Hasta 84 kg','Hasta 61.2 kg','Hasta 65.8 kg','Hasta 120.2 kg','Hasta 61.2 kg','Hasta 70.3 kg',
                            'Hasta 56.7 kg','Hasta 52.2 kg','Hasta 77.1 kg','Hasta 93 kg','Peso pactado',
                            'Más de 120.2 kg','Cualquier peso']}

division = pd.DataFrame(informacion)


In [165]:
#Creando DataFrame
division = pd.DataFrame(informacion)

In [166]:
#Guardando dataframe:
#division.to_csv("../data/Divisiones/Categorias_por_peso.csv")

# Algunas estadísticas rapidas
##### No serán objeto de estudio

In [167]:
#Donde se hacen mas eventos:
df_2["Donde"].value_counts()[:3]

Donde
Las Vegas, Nevada, USA                        1299
Abu Dhabi, Abu Dhabi, United Arab Emirates      83
London, England, United Kingdom                 78
Name: count, dtype: int64

In [168]:
#Quien ha participado mas veces:
df_2["Ganador"].value_counts()[:3]

Ganador
Charles Oliveira    20
Jim Miller          17
Donald Cerrone      16
Name: count, dtype: int64

In [169]:
#En que division ha habido mas combates:
df_2["Division"].value_counts()[:3]

Division
Peso Ligero    641
Peso Welter    626
Peso Medio     569
Name: count, dtype: int64

In [170]:
#Mas KO's o sumisiones:
df_2 = df_2[~df_2['Metodo'].str.contains("DQ", regex=True)]
df_2["Metodo"].value_counts()[:3]

Metodo
KO/TKO    2335
SUB       1479
Name: count, dtype: int64

In [171]:
#Round y tiempo mas veces visto:
frecuencias = df_2.groupby(['Round', 'Tiempo']).size().reset_index(name='count')
comun = frecuencias.loc[frecuencias['count'].idxmax()]
comun

Round      1.0
Tiempo    4:59
count       26
Name: 301, dtype: object

## Guardia



sfinasifngnizfsgnadnfgnkdzfgn
safobgaonfg



&nbsp; 
# **Proximamente...** ⏳ ⏳